# Import libraries

In [1]:
import pandas as pd
import numpy as np

# Enable interactive Matplotlib plots in the notebook
%matplotlib qt5


import matplotlib.pyplot as plt
from matplotlib import cm
import os
import astropy.convolution as krn
import scipy.stats as stats
import sys

import matplotlib.pyplot as plt
from matplotlib.widgets import Button



# Interactive Plotting Class

In [2]:
# class for interactive 2D plot

# Create a custom class to handle button clicks
class PlotUpdater:
    def __init__(self, ax, df, plot_type='2D'):
        self.ax = ax
        self.df = df
        self.plot_type = plot_type
        self.current_index = 0
        self.button_next = Button(ax, 'Next', color='lightgoldenrodyellow', hovercolor='0.975')
        self.button_prev = Button(ax, 'Previous', color='lightgoldenrodyellow', hovercolor='0.975')
        self.button_next.on_clicked(self.next_click)
        self.button_prev.on_clicked(self.prev_click)        
        
        # Set text color for the buttons to black
        self.button_next.label.set_color('black')
        self.button_prev.label.set_color('black')
        
    def next_click(self, event):
        if self.current_index < len(self.df.trialNr.unique()) - 1:
            self.current_index += 1
            self.update_plot()
        else:
            self.current_index = self.current_index - len(self.df.trialNr.unique()) # if at the end of df, wrap over to start of df
            self.update_plot()

    def prev_click(self, event):
        if self.current_index > 0:
            self.current_index -= 1
            self.update_plot()
        else:
            self.current_index = self.current_index + len(self.df.trialNr.unique())-1 # if at the start of df, wrap over to end of df
            self.update_plot()

    def update_plot(self):
        self.ax.clear()               
        
        # select a trial based on current_index
        current_trial = self.df.trialNr.unique()[self.current_index]
        df = self.df[self.df.trialNr == current_trial]
        
        if self.plot_type == '2D':        
            # plot 2D graph       
            self.plot2D(df, current_trial = current_trial)
        else:        
            # plot 1D graph
            self.plot1D(df, current_trial = current_trial)       

        
    def plot2D(self, df, current_trial):
        

        # plot raw samples
        self.ax.scatter(df['user_pred_px_x'], df['user_pred_px_y'], c='orange', alpha=0.5, edgecolors='black', label='raw samples')

        # plot fixations and remove no fixations/saccades (zeros)
        self.ax.scatter(df['FixXPos'][df['FixXPos']>0], df['FixYPos'][df['FixYPos']>0], c='blue', alpha=0.5, edgecolors='black', label='fixations')
        
#         # plot target and fixation cirle
#         self.ax.scatter(df.fixationStimX, df.fixationStimY, c='red')
#         self.ax.scatter(df.targetX, df.fixationStimY, c='green')
        
#         # plot target and fixation vertical lines
#         self.ax.plot(np.ones(df.resY.iloc[0].astype('int')) * df.fixationStimX.iloc[0], np.arange(df.resY.iloc[0]), c='red', lw=1, linestyle='dashed')
#         self.ax.plot(np.ones(df.resY.iloc[0].astype('int')) * df.targetX.iloc[0], np.arange(df.resY.iloc[0]), c='green', lw=1, linestyle='dashed')

        # set axis limits based on screen resolution
        self.ax.set_xlim((0, df.resX.iloc[0]))
        self.ax.set_ylim((df.resY.iloc[0]), 0)
        
        # label the axes
        self.ax.set_xlabel('Horizontal eye position (pixels)')
        self.ax.set_ylabel('Vertical eye position (pixels)')
        
        self.ax.legend()
        self.ax.set_title(f'Target {df.locStudiedImage.iloc[0]}, Trial {current_trial}')    
        self.ax.grid(True)
        
        plt.draw()
        
   

    def plot1D(self, df, current_trial):  
                     
        # rescale time variable to start from zero
        t = np.array(df.sampTime)
        t = t-t[0]
        
               
        # plot raw points and fixations
        self.ax.scatter(t, df.user_pred_px_x, c='orange', alpha=0.5, edgecolors='black', label='raw_samples')
        self.ax.scatter(t, df.FixXPos, c='blue', alpha=0.5, edgecolors='black', label='fixations')
        
        # add target and fixaton horizontal lines
        self.ax.plot(t, df.targetX, c='green', lw=1)
        self.ax.plot(t, df.fixationStimX, c='red', lw=1, linestyle='dashed')
        
        # label the axes
        self.ax.set_xlabel('Time (ms)')
        self.ax.set_ylabel('Horizontal eye position (pixels)')
        
        # set axis limits based on horizantal resolution
        self.ax.set_ylim((df.resX.iloc[0]), 0)
        
        # plot saccade latency as a horizontal line
        self.ax.plot(np.array([df.SaccLat.iloc[0]] * df.resX.iloc[0].astype(int)), np.arange(df.resX.iloc[0]), c='black', lw=1)
        
        
        # set the rest
        self.ax.set_title(f'Target {df.target.iloc[0]}, Trial {current_trial}')
        self.ax.legend()
        self.ax.grid(True)
        
        plt.draw()    

In [3]:
def makeHeat(screenRes, xPos, yPos):
        xMax = screenRes[0]
        yMax = screenRes[1]
        xMin = 0
        yMin = 0
        kernelPar = 50

        # Input handeling
        xlim = np.logical_and(xPos < xMax, xPos > xMin)
        ylim = np.logical_and(yPos < yMax, yPos > yMin)
        xyLim = np.logical_and(xlim, ylim)
        dataX = xPos[xyLim]
        dataX = np.floor(dataX)
        dataY = yPos[xyLim]
        dataY = np.floor(dataY)

        # initiate map and gauskernel
        gazeMap = np.zeros([int((xMax-xMin)),int((yMax-yMin))])+0.0001
        gausKernel = krn.Gaussian2DKernel(kernelPar)

        # Rescale the position vectors (if xmin or ymin != 0)
        dataX -= xMin
        dataY -= yMin

        # Now extract all the unique positions and number of samples
        xy = np.vstack((dataX, dataY)).T
        uniqueXY, idx, counts = uniqueRows(xy)
        uniqueXY = uniqueXY.astype(int)
        # populate the gazeMap
        gazeMap[uniqueXY[:,0], uniqueXY[:,1]] = counts

        # Convolve the gaze with the gauskernel
        heatMap = np.transpose(krn.convolve_fft(gazeMap,gausKernel))
        heatMap = heatMap/np.max(heatMap)

        return heatMap

def uniqueRows(x):
    y = np.ascontiguousarray(x).view(np.dtype((np.void, x.dtype.itemsize * x.shape[1])))
    _, idx, counts = np.unique(y, return_index=True, return_counts = True)
    uniques = x[idx]
    return uniques, idx, counts


def np_euclidean_distance(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    return np.sqrt(np.sum(np.square(y_pred - y_true), axis=-1))



# Preprocess, extract fixations and add them to the dataframe

In [4]:
   

def extract_fixations(df, path):
    
    sys.path.append('./FixationDetection')
    from I2MC import runI2MC
    
    # order frames and drop duplicate samples (with same sampleTime)
    df = df[df.fName.notna()]
    df.frameNr = df.frameNr.apply(pd.to_numeric, errors='coerce') # if framerNr is not a number, it is replaces with nan
    df = df[df.frameNr.notna()] # filter out rows where frameNr is a nan

    df = df[df.sampTime.notna()]
    df = df[df.user_pred_px_x.notna()]
    df = df[df.user_pred_px_y.notna()]
    df = df.apply(pd.to_numeric, errors='ignore') # if str convert str to numbers

    df = df.sort_values('frameNr')
    df = df.reset_index(drop=True)
    # df = df.drop_duplicates(subset=['user_pred_px_x', 'user_pred_px_y'], ignore_index=True)
    df = df.drop_duplicates(subset=['sampTime'], ignore_index=True)
       
    
    # get fixations for the original datafile for each participant
    fixDF = runI2MC(path, plotData = False)

    # add extracted fixations to the original data file (two new columns)
    # for each timestamp where fixation was detected, FixXPos and FixYPos are added
    idx = 0 # index of fixDF
    FixXPos = np.zeros(df.shape[0])
    FixYPos = np.zeros(df.shape[0])
    FixStartEnd = np.empty(df.shape[0], dtype='U10')
    FixStartEnd.fill('') # explicitly fill the array (good practice)
    FixDur = np.zeros(df.shape[0])

    DistFromPrevFix = np.zeros(df.shape[0])
    PrevFixXPos = np.zeros(df.shape[0])
    PrevFixYPos = np.zeros(df.shape[0])
    prev_fix_x = False # keep track of xy when fixation ends
    prev_fix_y = False

    PrevFixSampTime = np.zeros(df.shape[0])
    prev_fix_sampTime = 0

    # iterate thru the original dataframe, thru each sample
    for index, row in df.iterrows():

        # make sure not to iterate out of range
        if idx < fixDF.shape[0]:

            # go to next fixation when fixation ends
            if row['sampTime'] > np.array(fixDF.FixEnd)[idx]:
                    idx += 1

            # make sure not to iterate out of range
            if idx < fixDF.shape[0]:

                # when samples are within fixation, accumulate FixXPos and FixYPos
                if row['sampTime'] >= np.array(fixDF.FixStart)[idx] and row['sampTime'] <= np.array(fixDF.FixEnd)[idx]:

                    FixXPos[index] = (np.array(fixDF.XPos)[idx])
                    FixYPos[index] = (np.array(fixDF.YPos)[idx])

                # label samples on which fixation starts and ends
                if row['sampTime'] == np.array(fixDF.FixStart)[idx]:             
                    FixStartEnd[index] = 'fix_start'

                    if prev_fix_x != False:

                        PrevFixXPos[index] = prev_fix_x
                        PrevFixYPos[index] = prev_fix_y

                        DistFromPrevFix[index] = np.sqrt((np.array(fixDF.XPos)[idx] - prev_fix_x)**2 
                                                + (np.array(fixDF.YPos)[idx] - prev_fix_y)**2)
                        PrevFixSampTime[index] = prev_fix_sampTime


                elif row['sampTime'] == np.array(fixDF.FixEnd)[idx]:                
                    FixStartEnd[index] = 'fix_end' 
                    FixDur[index] = np.array(fixDF.FixDur)[idx]

                    prev_fix_x = np.array(fixDF.XPos)[idx]
                    prev_fix_y = np.array(fixDF.YPos)[idx]
                    prev_fix_sampTime = np.array(row['sampTime'])




    # add fixations to original dataframe
    df['FixXPos'] = np.array(FixXPos)
    df['FixYPos'] = np.array(FixYPos)
    df['FixStartEnd'] = FixStartEnd
    df['FixDur'] = np.array(FixDur)
    df['DistFromPrevFix'] = DistFromPrevFix
    df['PrevFixSampTime'] = PrevFixSampTime
    df['PrevFixXPos'] = PrevFixXPos
    df['PrevFixYPos'] = PrevFixYPos
    
    
    # Remove all negative xs, ys
    df = df[(df['FixXPos'] > 0) & (df['FixYPos'] > 0) & (df['user_pred_px_x'] > 0) & (df['user_pred_px_y'] > 0)]


    # Save the pre-processed dataframe
    df.to_csv((os.path.splitext(path)[0] + '_extra.csv'), index=False)  


    # Extract only samples when the target was presented
    df = df[df.event=='target_on']
    
    return df
    

# # # Label trials with too few data points
# # a = df.groupby('trialNr').count().reset_index()
# # a = a[['trialNr', 'sampTime']]
# # # 3) rename the columns so they would be added
# # a.columns = ['trialNr', 'samplesPerTrial']
# # df = pd.merge(df, a, on="trialNr")


# 2D plot of fixations and raw samples

## 1. plot all raw x,y
## 2. plot all fixations x,y 


In [30]:
def plot2d(df, subj_nr, path, bboxes=False, box_w=False, box_h=False, stimuli=False):
    
    import matplotlib.patches as patches
    import ast
    import matplotlib.image as mpimg

    conditions = ['bags', 'hats', 'sunglasses', 'shoes', 'dresses']


    for cond in conditions:

        plt.figure()
        plt.style.use('ggplot')
        plt.title(f'{cond}, #{subj_nr}')

        # select condition
        a = df[df.stimuli==cond] 
        
        if a.empty:
            # Raise an error or handle the case when the DataFrame is empty
            raise ValueError(f'There is no data for condition: {cond} in file: {subj_nr}')

        raw_h = plt.scatter(a.user_pred_px_x, a.user_pred_px_y, c='orange', alpha=0.5, edgecolors='black')

        # remove no fixations/saccades (zeros)
        
        fix_h = plt.scatter(a.FixXPos[a.FixXPos>0], a.FixYPos[a.FixYPos>0], c='blue', alpha=0.5, edgecolors='black') 

        # plot center fixation dot
        plt.scatter(a.resX.iloc[0]/2, a.resY.iloc[0]/2, c='red')      
              
        
        if bboxes:
            # plot image boxes
            if cond == 'dresses':
                height = 375
            else:
                width = 250
                height = 250
            
            # get coordinates for the first entry
            left_top_coords = a.imageLocations.iloc[0]
            
            # get paths to image names for that frame
            imageNames = a.imageNames.iloc[0]
            
            # Convert string to actual list
            left_top_coords = ast.literal_eval(left_top_coords)
#             print(f'coordinates: {left_top_coords}')
#             print(f'imageNames: {imageNames}')
            imageNames_paths = ast.literal_eval(imageNames)           
                        
            for stim_idx, coord in enumerate(left_top_coords):                
                left = float(coord[0])
                top = float(coord[1])            
               
                rect = patches.Rectangle((left,top),width,height, 
                                    fill = False,
                                    color = "purple",
                                    linewidth = 2)
                
                plt.gca().add_patch(rect)
                
                if stimuli:
#                     print('Plotting stimuli images') # draw stimuli images
                    
                    # Remove the last directory from the path
                    new_path = os.path.dirname(path)
                    
                    # Fix image name if ends on 'png'
                    if imageNames_paths[stim_idx][-3:] == 'png':
                        imageNames_paths[stim_idx] = imageNames_paths[stim_idx].strip('png') + 'jpg'
                    
                    # Join the main path with path to stimuli
                    new_path = new_path + imageNames_paths[stim_idx][1:]
                    
                    # Load the images
                    image = mpimg.imread(new_path)

                    # Define the extent (left, right, bottom, top) in data coordinates
                    extent = [left, left+width, top+height, top]
                    
                    # Plot the image at specific coordinates
                    plt.imshow(image, extent=extent)

                    
                    
                    


        plt.xlim((0, df.resX.iloc[0]))
        plt.ylim((df.resY.iloc[0]), 0)

        plt.xlabel('Horizontal eye position (pixels)')
        plt.ylabel('Vertical eye position (pixels)')

        plt.legend((raw_h, fix_h), ('raw samples', 'fixations'), scatterpoints=1)

        # save figure
        plt.savefig(os.path.join(path, subj_nr+f'_2D{cond}.jpg'), dpi=1000, pad_inches=0)
        
        plt.close()

    
    

# Get latency of all fixations and their order in a trial

In [6]:
# Get fixation latency

def getFixationLatency(df):    

    # Get timestamp of when target was presented and add it to the dataframe

    # 1) get the first time sample when the target is presented
    sampTime_df = df.drop_duplicates(subset=['trialNr'],  keep='first', ignore_index=True)

    # 2)extract the columns needed
    sampTime_df = sampTime_df[['trialNr', 'sampTime']]

    # 3) rename the columns so they would be added
    sampTime_df.columns = ['trialNr', 'targSampTime']

    # 4) merge the target time into the main df (one time per trial)
    df = pd.merge(df, sampTime_df, on="trialNr")

    # Extract saccade latencies

    # 1) select only rows where fixation started
    fl_df = df[df.FixStartEnd == 'fix_start']

    # 2) select only rows with large enough preceeding saccade
#     fl_df = fl_df[fl_df.DistFromPrevFix > 300]

    # 3) compute first fixation duration (saccade latency)
    fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime

    # 4) remove rows where negative Saccade Latencies for trials where no fixation end is present
#     fl_df = fl_df[fl_df.FixLatency > 0]

    # Clip the negative values to zero. This ensures that fixations that carry over and do not have fix_start have a zero latency
    fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)

    
    # 5)
    # Initialize an empty list to hold the groups
    fixorder_groups = []
    
    for name, group in fl_df.groupby('trialNr'):
        
        # Add a new column with the order (rank) of the values
        # 'method='first'' ensures that the order respects the original order in case of ties
        group['FixationOrder'] = group['FixLatency'].rank()
        
        # Append the modified group to the list
        fixorder_groups.append(group)
        
#         print(f'Group: {name}')
#         print(group)
#         print()        
    
    # Concatenate all the modified groups back into a single DataFrame
    fl_df_modified = pd.concat(fixorder_groups)
    
    # Extract the columns needed
    fl_df_modified = fl_df_modified[['sampTime', 'FixLatency', 'FixationOrder']]
    
    # Filter out all rows except fix_start and fix_end
    df_start_end = df[df['FixStartEnd'].isin(['fix_start', 'fix_end'])]

    # Merge the variable into main df
    df_modified = pd.merge(df_start_end, fl_df_modified, on=["sampTime"], how="left")
    
    return df_modified  
    
# df_modified = getFixationLatency(df1)




In [7]:
def handle_carryover_fixations_and_merge(df):

    trial_duration = 10000 # maximum trial duration
    fixcarryover_groups = []
    
    #***STEP 1: Identify, label and correct carryover fixations ***
    # Loop thru trials and identify fixations that carry over the trials
    # There are two options:
    # 1) Fixation starts during previous trial or event and ends at the beginning of the trial (fix_end event without fix_start event)
    # 2) Fixation starts at the end of current trial and finishes after the trial is done (fix_start event without fix_end event)
    # If not corrected, this results in incorrect fixation durations, fixation latency, missing fixations 
    for name, group in df.groupby('trialNr'):
        fix_start = group[group.FixStartEnd == 'fix_start'].FixStartEnd
        fix_end = group[group.FixStartEnd == 'fix_end'].FixStartEnd

        print(fix_start.count(), fix_end.count())

        # if trial starts with fixation end, we need to add a fixation start event
        if group.FixStartEnd.iloc[0] == 'fix_end':
            print(f'Trial {name} starts with fix_end')

            # Add extra fixation start event

            # Reset variable in the first row
            group.FixStartEnd.iloc[0] = 'fix_end_carryover_inserted_start' # label such fixation event
            group.FixDur.iloc[0] = group.sampTime.iloc[0] - group.targSampTime.iloc[0] # recalculate fixation duration

            # Insert a fix_start event
            # Make a copy of the first row with a new index
            first_row = group.iloc[0:1].copy()
            first_row.index = [-1]  # Assign a negative index
            first_row.FixStartEnd = 'fix_start_carryover_inserted_start'
            first_row.FixDur = 0
            first_row.FixLatency = 0


            # Prepend the copied first row to the original DataFrame
            group = pd.concat([first_row, group])

            # Reset the index if you want a continuous numeric index
            group = group.sort_index().reset_index(drop=True)

            # Now we need to re-rank the order of fixations in the trial, since we added a new one in the beginning
            group['FixationOrder'] = group['FixLatency'].rank()


        # if trial ends with fixation start, we need to add a fixation end event
        if group.FixStartEnd.iloc[-1] == 'fix_start':
            print(f'Trial {name} ends with fix_start')

            # Add fixation end event
            # Reset variable in the last row
            group.FixStartEnd.iloc[-1] = 'fix_start_carryover_inserted_end'
            group.FixDur.iloc[-1] = 0

            # Insert a fix_start event
            # Make a copy of the last row with a ne index
            last_row = group.iloc[[-1]].copy()        
            last_row.index = last_row.index+1 # Assign the next index
            last_row.FixStartEnd = 'fix_end_carryover_inserted_end'
            last_row.FixDur = (last_row.targSampTime + trial_duration) - last_row.sampTime
            last_row.FixLatency = 0
            last_row.FixationOrder= 0

            # Append the copied first row to the original DataFrame
            group = pd.concat([group, last_row])

            # Reset the index if you want a continuous numeric index
            group = group.sort_index().reset_index(drop=True)                      



        # Accumulate groups into a list
        fixcarryover_groups.append(group)

    # Concatenate all the modified groups back into a single DataFrame
    fc_df = pd.concat(fixcarryover_groups)

    #*** STEP 2: Collapse all fixation events, such that all information is provided per each fixation****
    
    # *** 1. Merge all fixation events for fixations that happen within trial ***
    
    # Get only fix_start events
    df_fix_start = fc_df[fc_df.FixStartEnd == 'fix_start']
    # Drop the FixDur column, which should be empty for fix_start events
    df_fix_start = df_fix_start.drop('FixDur', axis=1)

    # Get only fix_end events
    df_fix_end = fc_df[fc_df.FixStartEnd == 'fix_end']
    # Select only the relevant events from fix_end events
    df_fix_end = df_fix_end[["FixXPos", "FixYPos", "FixDur"]]


    # Merge fix start and end for the same fixations
    df_merged = pd.merge(df_fix_start, df_fix_end, on=["FixXPos", "FixYPos"])

    #*** 2. Merge carryover fixation which missed the fix_start event ***
    
    # Get only fix_start_carryover_inserted_start events
    df_fix_start_insert_start = fc_df[fc_df.FixStartEnd == 'fix_start_carryover_inserted_start']
    df_fix_start_insert_start = df_fix_start_insert_start.drop('FixDur', axis=1)

    # Get only fix_end_carryover_inserted_start events
    df_fix_end_insert_start = fc_df[fc_df.FixStartEnd == 'fix_end_carryover_inserted_start']
    # Select only the relevant events from fix_end events
    df_fix_end_insert_start = df_fix_end_insert_start[["FixXPos", "FixYPos", "FixDur"]]

    df_merged_insert_start = pd.merge(df_fix_start_insert_start, df_fix_end_insert_start, on=["FixXPos", "FixYPos"])

    #*** 3. Merge carryover fixations which missed the fix_end event ***
    
    # Get only fix_start_carryover_inserted_end events
    df_fix_start_insert_end = fc_df[fc_df.FixStartEnd == 'fix_start_carryover_inserted_end']
    df_fix_start_insert_end = df_fix_start_insert_end.drop('FixDur', axis=1)

    # Get only fix_end_carryover_inserted_end events
    df_fix_end_insert_end = fc_df[fc_df.FixStartEnd == 'fix_end_carryover_inserted_end']
    # Select only the relevant events from fix_end events
    df_fix_end_insert_end = df_fix_end_insert_end[["FixXPos", "FixYPos", "FixDur"]]

    df_merged_insert_end = pd.merge(df_fix_start_insert_end, df_fix_end_insert_end, on=["FixXPos", "FixYPos"])

    #*** 4. Concatenate all carryover fixations***
    # Now concatenate all carryover fixations, inserted_start and inserted_end
    df_carryover = pd.concat([df_merged_insert_start, df_merged_insert_end], ignore_index=True)

    # Sort the combined DataFrame based on frameNr
    df_carryover = df_carryover.sort_values(by='frameNr')

    
    # *** 5. Now concatenate carryover fixations with within_trial fixations ***
    df_final = pd.concat([df_merged, df_carryover], ignore_index=True)
    # Sort based on frameNr
    df_final = df_final.sort_values(by='frameNr')


    return df_final

# df_final = handle_carryover_fixations_and_merge(df_modified)    
    

# Calculating AOI for each fixation

In [9]:
def addAOI(df):  
    
    """
    df should contain only fixation events, no raw data
    """
    
    def is_point_in_box(point, box):
        """
        Determine if a point is within a bounding box.

        Parameters:
        - point: A tuple (x, y) representing the point.
        - box: A tuple ((x1, y1), (x2, y2)) representing the bounding box, 
               where (x1, y1) is the top-left corner and (x2, y2) is the bottom-right corner.

        Returns:
        - True if the point is within the box, False otherwise.
        """
        px, py = point
        (x1, y1), (x2, y2) = box

        return x1 <= px <= x2 and y1 <= py <= y2

    def get_bounding_box_assignment(boxes, point):
        """
        Determine the bounding box a point belongs to.

        Parameters:
        - boxes: A list of tuples representing the bounding boxes.
                 Each bounding box is defined as ((x1, y1), (x2, y2)).
        - point: A tuple (x, y) representing the point.

        Returns:
        - The index of the bounding box the point belongs to, or None if it doesn't belong to any boxes.
        """
        for i, box in enumerate(boxes):
            if is_point_in_box(point, box):
                return i
        
        return 'None'


    import ast
    width = 250
    height = 250
    imageName = 'None'
    bbox_assignments = []
    stim_assignments = []

    aoi_df = df

    for index, row in aoi_df.iterrows():
        
        if row.stimuli == 'dresses':
            height = 375            
        
        # Get all stimuli names for this trial
        imageNames = row.imageNames
        
        # Convert str to actual list
        imageNames = ast.literal_eval(imageNames)

        # Bounding boxes for this trial
        bounding_boxes = []

        # Get coordinates of stimuli for this fixation
        left_top_coords = row.imageLocations

        # Convert string to actual list
        left_top_coords = ast.literal_eval(left_top_coords)

        # Iterate over bboxes
        for coord in left_top_coords:      
            # Assemble coordinates for the bounding boxes
            x1 = coord[0]
            y1 = coord[1]
            x2 = coord[0] + width
            y2 = coord[1] + height
            bounding_boxes.append([(x1,y1), (x2,y2)])        

        # Get fixation coordinates
        point = (row.FixXPos, row.FixYPos)
        # get the index of the AOI where this fixation point falls
        assignment = get_bounding_box_assignment(bounding_boxes, point)

        # if fixation in the bounding box, get the stimulus name for this bounding box
        if assignment != 'None':
            imageName = imageNames[assignment].split('/')[-1]
        else:
            # To specify None, add the stimuli event 
            imageName = 'None_' + row.stimuli
            assignment = assignment + '_' + row.stimuli


#         print(f'The point {point} belongs to bounding box: {assignment}, image:{imageName}')


        # Accumulate assignements
        bbox_assignments.append(assignment)
        stim_assignments.append(imageName)

    aoi_df['AOI_bbox'] = bbox_assignments
    aoi_df['AOI_stim'] = stim_assignments
    
    # Reset index
    aoi_df = aoi_df.reset_index(drop=True)
       
    
    return aoi_df

# a = addAOI(df_final)
# a.to_csv(path +'/sample_participant_VBLExp1.csv', index=False)

# Analyze all subjects

In [31]:
# Path to data folders

path = 'C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved'
# path = 'D:/Dropbox/DeepEye_Pilots/VBL_Exp1/pilot_data/vbl_dressPilot_feb12/approved'
# path = 'C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/pilot_data/vbl_dressPilot_feb12/approved'

def create_directory_if_not_exists(directory_path):
    try:
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' was created.")
    except FileExistsError:
        # The directory already exists, no need to create it.
        print(f"Directory '{directory_path}' already exists.")

# definde data ana analysis directories and create them if they don't exist yet
path_to_data = os.path.join(path, 'data')
path_to_analysis = os.path.join(path, 'analysis')
create_directory_if_not_exists(path_to_analysis)

output_dfs = []
# get all folder names
folder_names = os.listdir(path_to_data)

# read and process original datafile for each participant
for fn in folder_names:
    path_to_file = os.path.join(path_to_data, fn, fn+'_record.csv')
    
    print(f'Processing participant {fn}...')
        
    try:
        df = pd.read_csv(path_to_file)
        print('File does not exist: ' + path_to_file)
    except:
        continue
        
    df1 = extract_fixations(df, path_to_file)
    
    try:
        plot2d(df1, fn, path_to_analysis, bboxes=True, stimuli=True)
    except ValueError:
        continue
        
    df1 = getFixationLatency(df1)
    df1 = handle_carryover_fixations_and_merge(df1)
    df1 = addAOI(df1)
    
    # Accumulate analyzed data across participants
    output_dfs.append(df1)
    
output_df = pd.concat(output_dfs)
output_df.to_csv(os.path.join(path_to_analysis, 'allSubjects_VBLExp1.csv'), index=False)



Directory 'C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\analysis' already exists.
Processing participant 2024_02_19_08_03_20...
File does not exist: C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_03_20\2024_02_19_08_03_20_record.csv



Importing and processing: "C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_03_20\2024_02_19_08_03_20_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 0.7119314670562744s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

18 19
Trial 0 starts with fix_end
15 15
12 13
Trial 2 starts with fix_end
14 15
Trial 3 starts with fix_end
16 17
Trial 4 starts with fix_end
Processing participant 2024_02_19_08_03_44...
File does not exist: C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_03_44\2024_02_19_08_03_44_record.csv



Importing and processing: "C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_03_44\2024_02_19_08_03_44_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 0.7179276943206787s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(low

16 17
Trial 0 starts with fix_end
15 15
Trial 1 starts with fix_end
Trial 1 ends with fix_start
19 19
13 14
Trial 3 starts with fix_end
18 18
Processing participant 2024_02_19_08_04_13...
File does not exist: C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_04_13\2024_02_19_08_04_13_record.csv



Importing and processing: "C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_02_19_08_04_13\2024_02_19_08_04_13_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 0.6939127445220947s to finish!


...

20 19
Trial 0 ends with fix_start
16 17
Trial 1 starts with fix_end
16 17
Trial 2 starts with fix_end
13 14
Trial 3 starts with fix_end
12 13
Trial 4 starts with fix_end
Processing participant 2024_03_15_08_53_54...
File does not exist: C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_03_15_08_53_54\2024_03_15_08_53_54_record.csv



Importing and processing: "C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_03_15_08_53_54\2024_03_15_08_53_54_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 1.2157323360443115s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(low

15 16
Trial 0 starts with fix_end
17 18
Trial 1 starts with fix_end
13 14
Trial 2 starts with fix_end
19 19
13 14
Trial 4 starts with fix_end
Processing participant 2024_03_15_11_15_54...
File does not exist: C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_03_15_11_15_54\2024_03_15_11_15_54_record.csv



Importing and processing: "C:/Users/artem/Dropbox/DeepEye_Pilots/VBL_Exp1/data/approved\data\2024_03_15_11_15_54\2024_03_15_11_15_54_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 0.6941144466400146s to finish!


C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df.sampTime - fl_df.targSampTime
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\1438454985.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fl_df['FixLatency'] = fl_df['FixLatency'].clip(lower=0)
C:\Users\artem\AppData\Local\Temp\ipykernel_20984\2621730654.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the ca

12 13
Trial 0 starts with fix_end
13 14
Trial 1 starts with fix_end
16 17
Trial 2 starts with fix_end
12 12
Trial 3 starts with fix_end
Trial 3 ends with fix_start
12 12


# Interactive 2D plot for every trial

In [8]:
# specify filename to plot interactively
# Path to data folders
path = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
# path_to_folders = 'D:/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_MullerLyer/pilot'

path_to_data = os.path.join(path, 'data')

fn = '2024_01_26_15_38_25' #'2024_01_26_17_16_28'
path_to_file = os.path.join(path_to_data, fn, fn+'_record.csv')       

# get fixations
df = pd.read_csv(path_to_file)
df1 = extract_fixations(df, path_to_file)

# Create a figure and initialize the plot
fig, ax = plt.subplots(figsize=(8, 6))
plot_updater = PlotUpdater(ax, df1, plot_type='2D')
plot_updater.update_plot()

# Position the 'Next' and 'Previous' buttons at the bottom of the plot
button_ax_prev = plt.axes([0.6, 0.01, 0.1, 0.05])
button_ax_next = plt.axes([0.8, 0.01, 0.1, 0.05])

plot_updater.button_next.ax = button_ax_next
plot_updater.button_prev.ax = button_ax_prev

plt.show()




Importing and processing: "C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved\data\2024_01_26_15_38_25\2024_01_26_15_38_25_record.csv"
	Searching for valid interpolation windows
	Replace interpolation windows with Steffen interpolation
	2-Means clustering started for averaged signal
	Determining fixations based on clustering weight mean for averaged signal and separate eyes + 2*std


I2MC took 3.3248491287231445s to finish!


C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\lib\nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
C:\Users\artem\anaconda3\envs\default\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


# Summary

In [25]:
path = 'C:/Users/artem/Dropbox/Appliedwork/CognitiveSolutions/Projects/DeepEye/TechnicalReports/TechnicalReport1/Test_PreferentialViewing/Pilot_PreferentialViewing/Young/Approved'
path_to_analysis = os.path.join(path, 'analysis')

output_df = pd.read_csv(os.path.join(path_to_analysis, 'allSubjects_NoveltyIndex.csv'))

# filter out excluded participants
# '2024_01_15_14_19_20' was the second time, '2024_01_26_17_16_28' too few frames
output_df = output_df[~output_df['deepeye-id'].isin(['2024_01_15_14_19_20', '2024_01_26_17_16_28'])]

# group data
total_count = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixCountProp.count()
fixCountProp = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixCountProp.mean()
fixDurProp = output_df.groupby(['pp_id', 'deepeye-id']).noveltyIdx_fixDurProp.mean()
